In [1]:
# This block gets list of TF numerical labels

import re, pandas as pd, numpy as np

f = open("JASPAR_web.html")
alltext = f.read()
f.close()
mytuples = re.findall(
    r'>\s*(\w+\.\w+)\s*&nbsp;\s*</td>\s*\n.*\n'
    '.*<!-.+->\n'
    '.*<td.+>\s*(\w+.*)\s*&nbsp;\s*</td>\s*'
    '\n.*\n.*<!-.+->\s*\n.*'
    '<td.+Homo sapiens.+</td>',
    alltext, re.MULTILINE)

filenames = [thing[0] for thing in mytuples]

In [2]:
#  Create column names for PSMM data frame
numsegments = 5

featurenames=[]
for file in filenames:
    for i in list(range(numsegments)):
        featurenames.append(file + '_' + str(i))

In [4]:
# This block gets list of gene labels and names

f = open("clover-2015-08-28/promresults.txt")
alltext = f.read()
f.close()
genes = re.findall(r'>(\w+)\s(\w+)\s+',alltext)

In [12]:
import numpy as np
import scipy as sp
from numpy import allclose, arange, eye, linalg, ones, random
import scipy.sparse.linalg.dsolve as linsolve
from scipy import sparse
import csv

g = open("PSMM_5seg2.csv",'w')
writer = csv.writer(g, delimiter=',')

# Copy from above
import timeit

numsegments = 5

start = timeit.default_timer()

#Your statements here

f = open("clover-2015-08-28/promresults.txt") #should be promresults (results from Clover run on promsequences.fa)

ind = 0

genevec = np.zeros(len(filenames)*numsegments, dtype=np.int)

# Read line by line
for line in f:


# This if statement checks if the line is not like "MA0081.1 SPIB   211 -  217" --a TF
    if re.findall(r'(\w+\.\w+)\s(\w+\(*.*\)*)\s+(\d+)\s+-\s+(\d+)',line)==[]:
# This if checks if the line is not like this: ">FP011394 KCTD12_1"---a new promoter
        if re.findall(r'>(\w+)\s(\w+)\s+',line)==[]:
# Must be a blank line (or not a line to be considered)
            pass
# Else could be ">FP011394 KCTD12_1" pattern---a new promoter (make sure)
        elif re.findall(r'>(\w+)\s(\w+)\s+',line)!=[]:            
            gene = re.findall(r'>(\w+)\s(\w+)\s+',line)[0]
            if ind >0:
                writer.writerow(genevec)
            genevec = np.zeros(len(filenames)*numsegments, dtype=np.int)
            ind += 1
        else:
            pass
# Else must be like "MA0081.1 SPIB   211 -  217"---the transcription factor result
    else:
        tf = re.findall(r'(\w+\.\w+)\s(\w+\(*.*\)*)\s+(\d+)\s+-\s+(\d+)',line)[0]
        tfid, tfname, x, y = tf
        tfcount = filenames.index(tfid)
        if ((1000//numsegments-int(x)%(1000//numsegments)) >= (int(y)%(1000//numsegments))):
            place = numsegments*tfcount + int(x)//(1000//numsegments)
            genevec[place] = 1
# To count, replace = above with +=
        else:
            place = numsegments*tfcount + int(y)//(1000//numsegments)
            genevec[place] = 1
# To count, replace = above with +=
writer.writerow(genevec)
f.close()

stop = timeit.default_timer()

print(stop - start)

98.5694928169
